## Capstone project, location for a new restaurent in Chicago, IL, USA

#### 1, Import Libraries

In [2]:
# import libraries section 
import numpy as np                        # library for vectorized computation
import pandas as pd                       # library to process data as dataframes

import requests                           # library to handle requests
from bs4 import BeautifulSoup             # library to parse HTML and XML documents

import json                               # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim     # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# install folium
!pip install folium

# import folium
import folium                              # map rendering library

print('Libraries imported!.')

     |████████████████████████████████| 94 kB 6.7 MB/s  eta 0:00:01
Libraries imported!.


#### 2, Import the Chicago neighborhood file chicagoneighborhood_2021.csv from google storage bucket.

In [4]:
# read csv file from google storage bucket.
##del df  # delete dataframe df before rerunning

df = pd.read_csv(r"https://storage.googleapis.com/natemano-courcera-chicago2021a/chicagoneighborhood_2021.csv", encoding = "ISO-8859-1")
df.head(6)

,Neighborhood,Community Area
0,Albany Park,Albany Park
1,Mayfair,Albany Park
2,North Mayfair,Albany Park
3,Ravenswood Manor,Albany Park
4,Archer Heights,Archer Heights
5,Armour Square,Armour Square


#### 3, Get geolocator codes for Chicago neighborhoods and build new dataframe

In [5]:
# define the dataframe columns
column_names = ['Community_Area', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the new dataframe
neighborhoods_chi = pd.DataFrame(columns=column_names)
neighborhoods_chi

,Community_Area,Neighborhood,Latitude,Longitude


In [6]:
# get Chicago neighborhood Latitude	Longitude
print('start read')

for idx, row in df.iterrows():

    neighborhood_name = row['Neighborhood']
    community_area = row['Community Area']
    
    address = neighborhood_name +', IL'
    #print('address ', address) # for debugging data
    
    geolocator = Nominatim(user_agent="chi_explorer")
    location   = geolocator.geocode(address)
    latitude   = location.latitude
    longitude  = location.longitude
    #print('out',location.latitude, location.longitude)  # for debugging data
     
    neighborhoods_chi = neighborhoods_chi.append({'Community_Area': community_area,
                                          'Neighborhood': neighborhood_name+(', IL'),
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)

#print('end read!')
neighborhoods_chi.head(6)

start read


,Community_Area,Neighborhood,Latitude,Longitude
0,Albany Park,"Albany Park, IL",41.971937,-87.716174
1,Albany Park,"Mayfair, IL",51.511087,-0.147058
2,Albany Park,"North Mayfair, IL",33.462919,-111.755610
3,Albany Park,"Ravenswood Manor, IL",41.964622,-87.701380
4,Archer Heights,"Archer Heights, IL",41.811422,-87.726165
5,Armour Square,"Armour Square, IL",41.840231,-87.632986


#### 4, create a map of chicago using latitude and longitude values. Use geopy library to get the latitude and longitude values of Chicago, IL

In [8]:
# Use geopy library to get the latitude and longitude values of Chicago, IL
print('start')
address = 'Chicago, IL'
geolocator    = Nominatim(user_agent="nm-application") # specify appliaction name
location      = geolocator.geocode(address)
latitude_chi  = location.latitude
longitude_chi = location.longitude
print('The geograpical coordinate of Chicago, IL are {}, {}.'.format(latitude_chi, longitude_chi))
print('done')

start
The geograpical coordinate of Chicago, IL are 41.8755616, -87.6244212.
done


In [9]:
#create a map of chicago using latitude and longitude values
print('start')
map_chicago = folium.Map(location=[latitude_chi, longitude_chi], zoom_start=10)

#add markers to map
#for borough, neighborhood, lat, lng in (neighborhoods_chi['Community Area'], neighborhoods_chi['Neighborhood'], neighborhoods_chi['Latitude'], neighborhoods_chi['Longitude']):
for borough, neighborhood, lat, lng in zip(neighborhoods_chi['Community_Area'], neighborhoods_chi['Neighborhood'], neighborhoods_chi['Latitude'], neighborhoods_chi['Longitude']):

    label = '{}, {}'.format(neighborhood, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chicago)  
    
map_chicago

start


#### 5, Foursquare Api for getting Nearby Venues

In [10]:
## Foursquare api set client account
CLIENT_ID = 'JXSHDM3INM2TUNT1Z3MVT2C3JUJC3LWVNOA1UTQ5A04BZEXZ' # your Foursquare ID
CLIENT_SECRET = 'T2YAIOY2B0YULJ1YJZXXTE4CCCAFZEVJ23XUQICT2AB4ZUA3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JXSHDM3INM2TUNT1Z3MVT2C3JUJC3LWVNOA1UTQ5A04BZEXZ
CLIENT_SECRET:T2YAIOY2B0YULJ1YJZXXTE4CCCAFZEVJ23XUQICT2AB4ZUA3


In [11]:
# Define function getNearbyVenues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### 6, Execute the above function on each neighborhood and create a new dataframe called chicago_venues.

In [12]:
# data from neighborhoods_chi

chicago_venues = getNearbyVenues(names=neighborhoods_chi['Neighborhood'],
                                   latitudes=neighborhoods_chi['Latitude'],
                                   longitudes=neighborhoods_chi['Longitude']
                                  )

Albany Park, IL
Mayfair, IL
North Mayfair, IL
Ravenswood Manor, IL
Archer Heights, IL
Armour Square, IL
Chinatown, IL
Wentworth Gardens, IL
Ashburn, IL
Ashburn Estates, IL
Beverly View, IL
Crestline, IL
Parkview, IL
Scottsdale, IL
Wrightwood, IL
Auburn Gresham, IL
Gresham, IL
Galewood, IL
North Austin, IL
South Austin, IL
The Island, IL
West Humboldt Park, IL
Avalon Park, IL
Marynook, IL
Stony Island Park, IL
Avondale, IL
Jackowo, IL
Polish Village, IL
Belmont Central, IL
Brickyard, IL
Cragin, IL
Hanson Park, IL
Beverly, IL
East Beverly, IL
West Beverly, IL
Bridgeport, IL
Brighton Park, IL
Burnside, IL
Calumet Heights, IL
Pill Hill, IL
Chatham, IL
East Chatham, IL
West Chatham, IL
West Chesterfield, IL
Chicago Lawn, IL
Lithuanian Plaza, IL
Marquette Park, IL
Chrysler Village, IL
Clearing East, IL
Clearing West, IL
Bronzeville, IL
Dearborn Homes, IL
Groveland Park, IL
Lake Meadows, IL
Prairie Shores, IL
South Commons, IL
Stateway Gardens, IL
The Gap, IL
Belmont Heights, IL
Belmont Terra

In [14]:
## check the size and head() of the resulting dataframe

print(chicago_venues.shape)
chicago_venues.head(11)

(3711, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Albany Park, IL",41.971937,-87.716174,Chicago Produce,41.970553,-87.716327,Grocery Store
1,"Albany Park, IL",41.971937,-87.716174,Cairo Nights Hookah Lounge,41.975776,-87.715547,Hookah Bar
2,"Albany Park, IL",41.971937,-87.716174,Nighthawk,41.967974,-87.713415,Cocktail Bar
3,"Albany Park, IL",41.971937,-87.716174,Peking Mandarin Resturant,41.968292,-87.715783,Chinese Restaurant
4,"Albany Park, IL",41.971937,-87.716174,Popeyes Louisiana Kitchen,41.968756,-87.713019,Fried Chicken Joint
5,"Albany Park, IL",41.971937,-87.716174,Hiromi's Oriental Restaurant,41.968144,-87.718719,Karaoke Bar
6,"Albany Park, IL",41.971937,-87.716174,Markellos Baking Company,41.968602,-87.716607,Bakery
7,"Albany Park, IL",41.971937,-87.716174,Banpojung,41.975707,-87.715609,Korean Restaurant
8,"Albany Park, IL",41.971937,-87.716174,Subway,41.968748,-87.712861,Sandwich Place
9,"Albany Park, IL",41.971937,-87.716174,T-Mobile,41.968751,-87.713158,Mobile Phone Shop


In [15]:
## Check number of venues returned for each neighborhood
chicago_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Albany Park, IL",15,15,15,15,15,15
"Altgeld Gardens, IL",3,3,3,3,3,3
"Andersonville, IL",4,4,4,4,4,4
"Archer Heights, IL",19,19,19,19,19,19
"Armour Square, IL",13,13,13,13,13,13
...,...,...,...,...,...,...
"Wicker Park, IL",50,50,50,50,50,50
"Wildwood, IL",23,23,23,23,23,23
"Woodlawn, IL",1,1,1,1,1,1


In [16]:
## check new dataframe size.
chicago_venues.shape

(3711, 7)

### 7, Check the number of unique categories that can be curated from all the returned venues.

In [19]:
## Check number of unique categories that can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 373 uniques categories.


### 8, Let's Analyze each neighborhood

In [21]:
## Let's Analyze Each Neighborhood
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head(20)

,Zoo,ATM,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
## let's analyze the new dataframe size.
chicago_onehot.shape

(3711, 373)

### 9, Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
##Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_grouped

,Neighborhood,Zoo,ATM,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Albany Park, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,"Altgeld Gardens, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,"Andersonville, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,"Archer Heights, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.052632,0.0,0.0
4,"Armour Square, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,"Wicker Park, IL",0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
213,"Wildwood, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.000000,0.0,0.0
214,"Woodlawn, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
215,"Wrightwood, IL",0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [25]:
## Lets confirm the new size

chicago_grouped.shape

(217, 373)

### 10, Print each neighborhood along with the top 5 most common venues

In [26]:
# set top venues to 5 & Print each neighborhood along with common venues
num_top_venues = 5

for hood in chicago_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park, IL----
                 venue  freq
0        Grocery Store  0.20
1         Cocktail Bar  0.07
2    Korean Restaurant  0.07
3  Fried Chicken Joint  0.07
4               Bakery  0.07


----Altgeld Gardens, IL----
           venue  freq
0           Park  0.33
1           Food  0.33
2  Grocery Store  0.33
3            Zoo  0.00
4         Office  0.00


----Andersonville, IL----
                 venue  freq
0       History Museum  0.50
1   Travel & Transport  0.25
2  American Restaurant  0.25
3                  Zoo  0.00
4               Office  0.00


----Archer Heights, IL----
                venue  freq
0       Grocery Store  0.11
1  Mexican Restaurant  0.11
2   Mobile Phone Shop  0.11
3                Bank  0.05
4       Hot Dog Joint  0.05


----Armour Square, IL----
                venue  freq
0  Chinese Restaurant  0.23
1       Hot Dog Joint  0.08
2       Grocery Store  0.08
3      Sandwich Place  0.08
4       Tanning Salon  0.08


----Ashburn Estates, IL----
         

### 11, Create the new dataframe and display the top 7 venues for each neighborhood.
#### define function return_most_common_venues and create dataframe neighborhoods_venues_sorted.

In [28]:
## define function return_most_common_venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
## create the new dataframe and display the top 7 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = chicago_grouped['Neighborhood']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(6)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albany Park, IL",Grocery Store,Fried Chicken Joint,Korean Restaurant,Chinese Restaurant,Cocktail Bar,Latin American Restaurant,Bus Station,Donut Shop,Karaoke Bar,Mobile Phone Shop
1,"Altgeld Gardens, IL",Food,Grocery Store,Park,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
2,"Andersonville, IL",History Museum,Travel & Transport,American Restaurant,Yoga Studio,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
3,"Archer Heights, IL",Grocery Store,Mobile Phone Shop,Mexican Restaurant,Ice Cream Shop,Big Box Store,Bakery,Sandwich Place,Candy Store,Chinese Restaurant,Gym / Fitness Center
4,"Armour Square, IL",Chinese Restaurant,Grocery Store,Italian Restaurant,Tanning Salon,Asian Restaurant,Gas Station,Café,Sandwich Place,Hot Dog Joint,Cosmetics Shop
5,"Ashburn Estates, IL",Home Service,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit


### 12, Cluster Neighborhoods
### Run k-means to cluster the neighborhood into 5 clusters.

In [71]:
##Find K


chicago_grouped.head()

,Neighborhood,Zoo,ATM,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Albany Park, IL",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Altgeld Gardens, IL",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Andersonville, IL",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,"Archer Heights, IL",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0
4,"Armour Square, IL",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [31]:
# set number of clusters fit(chicago_grouped_clustering)
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 1, 4, 4, 2, 4, 0, 4, 4], dtype=int32)

### 13, Add cluster lables

In [38]:
### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#chicago_merged = chicago_data
chicago_merged = neighborhoods_chi

# merge chicago_grouped with neighborhoods_chi to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

chicago_merged.head() # check the last columns!

,Community_Area,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,"Albany Park, IL",41.971937,-87.716174,4.0,Grocery Store,Fried Chicken Joint,Korean Restaurant,Chinese Restaurant,Cocktail Bar,Latin American Restaurant,Bus Station,Donut Shop,Karaoke Bar,Mobile Phone Shop
1,Albany Park,"Mayfair, IL",51.511087,-0.147058,4.0,Hotel,Boutique,Clothing Store,Art Gallery,French Restaurant,Steakhouse,Park,Juice Bar,Seafood Restaurant,Lounge
2,Albany Park,"North Mayfair, IL",33.462919,-111.755610,4.0,Pool,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
3,Albany Park,"Ravenswood Manor, IL",41.964622,-87.701380,4.0,Train Station,Park,Peruvian Restaurant,Pie Shop,Museum,Coffee Shop,Health & Beauty Service,Indoor Play Area,Laundromat,Video Game Store
4,Archer Heights,"Archer Heights, IL",41.811422,-87.726165,4.0,Grocery Store,Mobile Phone Shop,Mexican Restaurant,Ice Cream Shop,Big Box Store,Bakery,Sandwich Place,Candy Store,Chinese Restaurant,Gym / Fitness Center


### 24, Final - visualize the resulting clusters

In [114]:
## Final - visualize the resulting clusters using folium

# create map Chicago
map_clusters  = folium.Map(location=[latitude_chi, longitude_chi], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels'].fillna(4)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
   # print((chicago_merged['Cluster Labels'].fillna(0).unique()))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
#### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

In [104]:
# Cluster 1
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"Auburn Gresham, IL",Park,Discount Store,Basketball Court,Yoga Studio,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
38,"Calumet Heights, IL",Pharmacy,Park,Gym / Fitness Center,Deli / Bodega,Bus Station,Yoga Studio,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
46,"Marquette Park, IL",Park,Liquor Store,Insurance Office,Soccer Field,Yoga Studio,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
49,"Clearing West, IL",Airport Terminal,Park,Airport,Athletics & Sports,Yoga Studio,Farm,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
54,"Prairie Shores, IL",Park,Shopping Mall,Bus Station,Gym / Fitness Center,Train Station,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
55,"South Commons, IL",Park,Gym,Gym / Fitness Center,Rental Car Location,Fabric Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room
60,"Dunning, IL",Park,Intersection,Deli / Bodega,Construction & Landscaping,Flower Shop,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room
77,"Sauganash, IL",Park,Pharmacy,Indian Restaurant,Asian Restaurant,Fast Food Restaurant,Basketball Court,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room
80,"Fuller Park, IL",Park,Sandwich Place,Intersection,Fast Food Restaurant,Chinese Restaurant,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
85,"Vittum Park, IL",Park,Mexican Restaurant,Furniture / Home Store,Discount Store,Yoga Studio,Farm,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space


In [106]:
## Cluster 2
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,"Chicago Lawn, IL",Pizza Place,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Fish & Chips Shop,Intersection,Transportation Service,Falafel Restaurant,Empanada Restaurant,English Restaurant
52,"Groveland Park, IL",Convenience Store,Airport,Music Venue,Farmers Market,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
63,"East Garfield Park, IL",Pharmacy,Pet Service,Diner,Warehouse Store,Burger Joint,American Restaurant,Train Station,Garden Center,Bus Line,English Restaurant
66,"Andersonville, IL",History Museum,Travel & Transport,American Restaurant,Yoga Studio,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
75,"Forest Glen, IL",Pharmacy,Pool,Playground,Intersection,Yoga Studio,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant
83,"LeClaire Courts, IL",Intersection,Botanical Garden,Farmers Market,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
89,"Greater Grand Crossing, IL",Currency Exchange,American Restaurant,Convenience Store,Discount Store,Flower Shop,Fast Food Restaurant,Seafood Restaurant,Liquor Store,Candy Store,Food
95,"Humboldt Park, IL",Gas Station,Latin American Restaurant,American Restaurant,Yoga Studio,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
129,"Palmer Square, IL",Fast Food Restaurant,Food,Southern / Soul Food Restaurant,Fried Chicken Joint,Supermarket,Pizza Place,American Restaurant,Falafel Restaurant,Empanada Restaurant,English Restaurant
170,"North Lawndale, IL",Convenience Store,Concert Hall,Food,Farm,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit


In [107]:
## Cluster 3
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Ashburn Estates, IL",Home Service,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
173,"River's Edge, IL",Home Service,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit


In [108]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
111,"Lake View East, IL",Resort,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
195,"Princeton Park, IL",Gym,Baseball Field,Resort,Yoga Studio,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant


In [110]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4, chicago_merged.columns[[1] + list(range(5, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albany Park, IL",Grocery Store,Fried Chicken Joint,Korean Restaurant,Chinese Restaurant,Cocktail Bar,Latin American Restaurant,Bus Station,Donut Shop,Karaoke Bar,Mobile Phone Shop
1,"Mayfair, IL",Hotel,Boutique,Clothing Store,Art Gallery,French Restaurant,Steakhouse,Park,Juice Bar,Seafood Restaurant,Lounge
2,"North Mayfair, IL",Pool,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
3,"Ravenswood Manor, IL",Train Station,Park,Peruvian Restaurant,Pie Shop,Museum,Coffee Shop,Health & Beauty Service,Indoor Play Area,Laundromat,Video Game Store
4,"Archer Heights, IL",Grocery Store,Mobile Phone Shop,Mexican Restaurant,Ice Cream Shop,Big Box Store,Bakery,Sandwich Place,Candy Store,Chinese Restaurant,Gym / Fitness Center
...,...,...,...,...,...,...,...,...,...,...,...
234,"Smith Park, IL",Thai Restaurant,Yoga Studio,Farm,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
235,"Ukrainian Village, IL",Pub,Dive Bar,Coffee Shop,Ukrainian Restaurant,Eastern European Restaurant,Grocery Store,Bar,Art Museum,Bank,Pizza Place
236,"West Town, IL",Dive Bar,Grocery Store,Mexican Restaurant,Bar,Taco Place,Toy / Game Store,Chinese Restaurant,Bank,Latin American Restaurant,Eastern European Restaurant
237,"Wicker Park, IL",Pizza Place,Coffee Shop,Bookstore,Bar,Accessories Store,Breakfast Spot,Boutique,Sporting Goods Shop,Korean Restaurant,New American Restaurant
